In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
# torch.tensor(data) создает объект torch.Tensor с заданными данными.
vector_list = [1., 2., 3.]
vector = torch.tensor(vector_list)
print(vector)

# Создает матрицу
matrix_list = [[1., 2., 3.], [4., 5., 6]]
matrix = torch.tensor(matrix_list)
print(matrix)

# Создает 3D тензор размера 2x2x2.
tensor_list = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
tensor = torch.tensor(tensor_list)
print(tensor)

In [ ]:
# По индексу из вектора можно получить значение в виде вектора (0 dimensional тензор)
print(vector[0])
# Получить python число
print(vector[0].item())

# По индексу из матрицы мы получим вектор
print(matrix[0])

# По индексу из тензора мы получим матрицу
print(tensor[0])

Вы также можете создавать тензоры других типов данных. Чтобы создать тензор целочисленных типов, попробуйте `torch.tensor([[1, 2], [3, 4]])` (где все элементы должны быть типа integer). Вы также можете указать тип данных, передав `dtype=torch.data_type`. Обратитесь к документации, чтобы узнать о других типах данных, но наиболее распространенными будут Float и Long.

Вы можете создать тензор со случайными данными и заданной размерностью с помощью `torch.randn()`.

In [ ]:
x = torch.randn((3, 4, 5))
print(x)

## Операции с тензорами

In [ ]:
x = torch.tensor([1., 2., 3.])
y = torch.tensor([4., 5., 6.])
z = x + y
print(z)

В [документации](https://pytorch.org/docs/stable/torch.html) можно найти полный список доступных операций. Они выходят за рамки просто математических операций.

Одна полезная операция, которую мы будем использовать позже - это конкатенация.

In [ ]:
# По умолчанию он конкатенирует по первой оси (объединяет строки)
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 = torch.cat([x_1, y_1])
print(z_1)

# Конкатенация колонок:
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
# second arg specifies which axis to concat along
z_2 = torch.cat([x_2, y_2], 1)
print(z_2)

# Если ваши тензоры несовместимы, torch выкинет ошибку. Раскомментируйте, чтобы увидеть ошибку
# torch.cat([x_1, x_2])

## Изменение формы тензоров

Используйте .view() метод для изменения формы тензоров. Этот метод широко используется, потому что многие компоненты нейронной сети ожидают, что их входные данные будут иметь определенную форму. Часто вам нужно изменить форму перед передачей данных в компонент.

In [ ]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12))  # Изменение формы в 2 строки и 12 колонок
# Аналогичная операция ниже. Если одна из размерностей равна -1, то размер другой можно подобрать автоматически
print(x.view(2, -1))

## Граф вычислений и автоматическая дифференациация

Концепция графа вычислений важна для эффективного программирования с глубоким обучением, поскольку позволяет не писать градиенты обратного распространения самостоятельно. Граф вычислений - это просто спецификация того, как ваши данные объединяются, чтобы дать вам результат. Поскольку граф полностью определяет, какие параметры были задействованы в каких операциях, он содержит достаточно информации для вычисления производных. Возможно, это звучит расплывчато, поэтому давайте посмотрим, что происходит, используя основной флаг `requires_grad`.

Во-первых, подумайте с точки зрения программиста. Что хранится в объектах torch.Tensor, которые мы создавали выше? Очевидно, данные и форма, и, возможно, еще несколько вещей. Но когда мы сложили два тензора вместе, мы получили выходной тензор. Все, что знает этот выходной тензор, - это его данные и форма. Он понятия не имеет, что это была сумма двух других тензоров (она могла быть прочитана из файла, могла быть результатом какой-то другой операции и т. Д.)

Если `requires_grad=True`, объект Tensor отслеживает, как он был создан. Давайте посмотрим на это в действии.

In [ ]:
# У фабричного метода Tensor есть флаг ``requires_grad``
x = torch.tensor([1., 2., 3], requires_grad=True)

# С requires_grad=True, вы по-прежнему можете выполнять все операции, которые вы ранее могли проводить
y = torch.tensor([4., 5., 6], requires_grad=True)
z = x + y
print(z)

# Но z знает немного больше
print(z.grad_fn)

Значит, тензоры знают, что их создало. z знает, что это не было прочитано из файла, это не было результатом умножения, экспоненты или чего-то еще. И если вы продолжите следовать z.grad_fn, вы окажетесь в x и y.

Но как это помогает нам вычислить градиент?

In [ ]:
# Давайте просуммируем все значения в z
s = z.sum()
print(s)
print(s.grad_fn)


Итак, какова производная этой суммы по первой компоненте x? Исходя из математики мы хотим

$$\frac{\partial s}{\partial x_0}$$

Что ж, s знает, что он был создан как сумма тензора z. z знает, что это была сумма x + y. Так

$$s=x_0+y_0 + x_1+y_1 + x_2+y_2$$

где $x_0 + y_0 = z_0$ и т.д.

Таким образом, s содержит достаточно информации, чтобы определить, что нам нужна производная 1!

Конечно, при этом игнорируется проблема того, как на самом деле вычислить эту производную. Дело здесь в том, что s несет достаточно информации, чтобы ее можно было вычислить. На самом деле разработчики Pytorch программируют операции sum() и +, чтобы знать, как вычислять их градиенты, и запускать алгоритм обратного распространения. Подробное обсуждение этого алгоритма выходит за рамки этого туториала.

Пусть Pytorch вычислит градиент и убедится, что мы были правы: (обратите внимание, если вы запустите этот блок несколько раз, градиент будет увеличиваться. Это потому, что Pytorch накапливает градиент в свойстве .grad, поскольку для многих моделей это очень удобно .)

In [ ]:
# calling .backward() on any variable will run backprop, starting from it.
s.backward()
print(x.grad)